### Depp Convolutional Neural Network

In this example, we are going to construct a network that looks like:

![Convolutional neural network](../images/mnist-convnet.png)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

### The Network class

In [137]:
class Network(nn.Module):
    
    def __init__(self, **kwargs):
        super(Network, self).__init__()
        
        # Hyper-parameters
        self.lr = kwargs.get('lr', 1e-2)
        self.max_iter = kwargs.get('max_iter', 10000)
        self.img_channels = kwargs.get('img_channels', 1)
        self.num_classes = kwargs.get('num_classes', 10)
        
        # 2 convolutional & 3 fully connected layers
        self.conv1 = nn.Conv2d(self.img_channels, 6, 1)
        self.conv2 = nn.Conv2d(6, 16, 1)
        flatten_size = self.conv2.out_channels * 8 * 8
        self.fc1 = nn.Linear(flatten_size, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, self.num_classes)
    
    def forward(self, x):
        # Convolutional layers
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        # Flatten layer
        x = x.view(-1, self._flatten(x))
        # Fully connected layers
        x = F.relu(self.fc1(x))     # relu + linear
        x = F.dropout(x, p=0.5)     # 50% dropout
        x = F.relu(self.fc2(x))     # relu + linear
        x = F.sigmoid(self.fc3(x))  # sigmoid + linear
        return x
    
    def _flatten(self, x):
        size = x.size()[1:]  # input shape excluding batch dim.
        return torch.Tensor(size).numel()

### importing the data

In [138]:
net = Network()

print(net)

Network(
  (conv1): Conv2d (1, 6, kernel_size=(1, 1), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=1024, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)


In [140]:
# Input image
X_image = Variable(torch.randn(1, 1, 32, 32))
y_true = torch.zeros(1, 10)
y_true[]
y_true = Variable(y_true)
# Net prediction
y_pred = net(X_image)
print(y_true)

Variable containing:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
[torch.FloatTensor of size 10]

